In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from sklearn.preprocessing import MultiLabelBinarizer
from PIL import Image
from sklearn.metrics import precision_recall_curve

# Конфигурация
DATASET_PATH = 'subcategory_images\dataset'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30  # Увеличили количество эпох
LEARNING_RATE = 0.0005  # Уменьшили learning rate
MODEL_SAVE_PATH = 'multi_label_classifier_improved.h5'

# Сопоставление подкатегорий с основными категориями
SUBCAT_TO_CAT = {
    'ak': 'Страйкбольное оружие',
    'backpack': 'Снаряжение и защита',
    'helmet': 'Снаряжение и защита',
    'HK': 'Страйкбольное оружие',
    'M serias': 'Страйкбольное оружие',
    'mashinegun': 'Страйкбольное оружие',
    'pistol': 'Страйкбольное оружие',
    'pouch': 'Аксессуары и Запчасти',
    'rifle': 'Страйкбольное оружие',
    'shutgun': 'Страйкбольное оружие',
    'vest': 'Снаряжение и защита'
}

# Получаем список всех подкатегорий и категорий
subcategories = sorted(list(SUBCAT_TO_CAT.keys()))
categories = sorted(list(set(SUBCAT_TO_CAT.values())))

# Инициализация MultiLabelBinarizer
mlb_subcat = MultiLabelBinarizer()
mlb_subcat.fit([subcategories])
mlb_cat = MultiLabelBinarizer()
mlb_cat.fit([categories])

# Аугментация данных
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
])

def load_and_preprocess_data(dataset_path):
    image_paths = []
    subcat_labels = []
    cat_labels = []
    
    for subcat in subcategories:
        subcat_path = os.path.join(dataset_path, subcat)
        if not os.path.exists(subcat_path):
            continue
            
        cat = SUBCAT_TO_CAT[subcat]
        for image_name in os.listdir(subcat_path):
            image_path = os.path.join(subcat_path, image_name)
            image_paths.append(image_path)
            subcat_labels.append([subcat])
            cat_labels.append([cat])
    
    subcat_labels = mlb_subcat.transform(subcat_labels)
    cat_labels = mlb_cat.transform(cat_labels)
    
    return image_paths, subcat_labels, cat_labels

# Загрузка данных
image_paths, subcat_labels, cat_labels = load_and_preprocess_data(DATASET_PATH)

# Создание tf.data.Dataset с аугментацией
def process_data(item):
    image = tf.io.read_file(item['image_path'])
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = data_augmentation(image)  # Применяем аугментацию
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, (item['subcat_label'], item['cat_label'])

dataset = tf.data.Dataset.from_tensor_slices({
    'image_path': image_paths,
    'subcat_label': subcat_labels,
    'cat_label': cat_labels
})

dataset = dataset.map(process_data, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=len(image_paths))
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Разделение на train/validation
train_size = int(0.8 * len(image_paths))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Обновленная версия f1_score
def f1_score(y_true, y_pred, threshold=0.5):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    y_true = tf.cast(y_true, tf.float32)  # Добавляем приведение типа
    tp = tf.reduce_sum(y_true * y_pred)
    precision = tp / (tf.reduce_sum(y_pred) + 1e-7)
    recall = tp / (tf.reduce_sum(y_true) + 1e-7)
    return 2 * (precision * recall) / (precision + recall + 1e-7)

# Создание модели с улучшенной архитектурой
def create_model(num_subcategories, num_categories):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))
    
    # Замораживаем первые 150 слоев
    for layer in base_model.layers[:150]:
        layer.trainable = False
    
    inputs = Input(shape=(*IMAGE_SIZE, 3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    
    # Улучшенная головная часть
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    subcat_output = Dense(num_subcategories, activation='sigmoid', name='subcat_output')(x)
    cat_output = Dense(num_categories, activation='sigmoid', name='cat_output')(x)
    
    model = Model(inputs=inputs, outputs=[subcat_output, cat_output])
    
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss={
            'subcat_output': 'binary_crossentropy',
            'cat_output': 'binary_crossentropy'
        },
        metrics={
            'subcat_output': ['accuracy', f1_score],
            'cat_output': ['accuracy', f1_score]
        }
    )
    
    return model

model = create_model(len(subcategories), len(categories))
model.summary()

# Коллбэки
callbacks = [
    ModelCheckpoint(
        MODEL_SAVE_PATH, 
        save_best_only=True, 
        monitor='val_subcat_output_f1_score', 
        mode='max'
    ),
    EarlyStopping(
        patience=7, 
        restore_best_weights=True, 
        monitor='val_subcat_output_f1_score',
        mode='max'  # Явно указываем, что хотим максимизировать F1-score
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=3, 
        min_lr=1e-6,
        mode='min'  # Для val_loss используем минимизацию
    )
]

# Обучение
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Функция предсказания с TTA (Test-Time Augmentation)
def predict_with_tta(image_path, model, tta_steps=5, threshold=0.5):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(IMAGE_SIZE)
    img_array = np.array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    
    subcat_preds = []
    cat_preds = []
    
    for _ in range(tta_steps):
        augmented_img = data_augmentation(img_array)
        augmented_img = np.expand_dims(augmented_img, axis=0)
        subcat_pred, cat_pred = model.predict(augmented_img)
        subcat_preds.append(subcat_pred)
        cat_preds.append(cat_pred)
    
    subcat_pred_avg = np.mean(subcat_preds, axis=0)
    cat_pred_avg = np.mean(cat_preds, axis=0)
    
    subcat_indices = np.where(subcat_pred_avg[0] > threshold)[0]
    cat_indices = np.where(cat_pred_avg[0] > threshold)[0]
    
    predicted_subcats = mlb_subcat.classes_[subcat_indices]
    predicted_cats = mlb_cat.classes_[cat_indices]
    
    subcat_probs = {subcat: float(subcat_pred_avg[0][i]) for i, subcat in enumerate(mlb_subcat.classes_)}
    cat_probs = {cat: float(cat_pred_avg[0][i]) for i, cat in enumerate(mlb_cat.classes_)}
    
    return {
        'categories': list(predicted_cats),
        'subcategories': list(predicted_subcats),
        'category_probabilities': cat_probs,
        'subcategory_probabilities': subcat_probs
    }

# Пример использования
if __name__ == "__main__":
    model = tf.keras.models.load_model(MODEL_SAVE_PATH, custom_objects={'f1_score': f1_score})
    result = predict_with_tta('тест\дроб3.jpeg', model)
    print(result)

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
C:\Users\AI-DEL\AppData\Local\Temp\ipykernel_1984\3607511371.py:15: SyntaxWarning: invalid escape sequence '\d'
  DATASET_PATH = 'subcategory_images\dataset'


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ input_layer_2[0]… │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │    655,872 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subcat_output       │ (None, 11)        │      5,643 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_output (Dense)  │ (None, 3)         │      1,539 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,714,673 (17.99 MB)

 Trainable params: 3,951,782 (15.07 MB)

...

29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - cat_output_accuracy: 0.9969 - cat_output_f1_score: 0.9963 - cat_output_loss: 0.0114 - loss: 0.0348 - subcat_output_accuracy: 0.9921 - subcat_output_f1_score: 0.9758 - subcat_output_loss: 0.0234 - learning_rate: 5.0000e-04
Epoch 28/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - cat_output_accuracy: 0.9987 - cat_output_f1_score: 0.9986 - cat_output_loss: 0.0042 - loss: 0.0218 - subcat_output_accuracy: 0.9951 - subcat_output_f1_score: 0.9890 - subcat_output_loss: 0.0176

29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - cat_output_accuracy: 0.9987 - cat_output_f1_score: 0.9986 - cat_output_loss: 0.0042 - loss: 0.0218 - subcat_output_accuracy: 0.9951 - subcat_output_f1_score: 0.9889 - subcat_output_loss: 0.0176 - learning_rate: 5.0000e-04
Epoch 29/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - cat_output_accuracy: 0.9968 - cat_output_f1_score: 0.9978 - cat_output_loss: 0.0079 - loss: 0.0270 - subcat_output_accuracy: 0.9944 - subcat_output_f1_score: 0.9849 - subcat_output_loss: 0.0190

29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - cat_output_accuracy: 0.9969 - cat_output_f1_score: 0.9978 - cat_output_loss: 0.0078 - loss: 0.0268 - subcat_output_accuracy: 0.9944 - subcat_output_f1_score: 0.9849 - subcat_output_loss: 0.0190 - learning_rate: 5.0000e-04
Epoch 30/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - cat_output_accuracy: 1.0000 - cat_output_f1_score: 0.9958 - cat_output_loss: 0.0112 - loss: 0.0278 - subcat_output_accuracy: 0.9963 - subcat_output_f1_score: 0.9861 - subcat_output_loss: 0.0166

29/29 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - cat_output_accuracy: 1.0000 - cat_output_f1_score: 0.9959 - cat_output_loss: 0.0111 - loss: 0.0277 - subcat_output_accuracy: 0.9962 - subcat_output_f1_score: 0.9859 - subcat_output_loss: 0.0166 - learning_rate: 5.0000e-04


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
{'categories': ['Страйкбольное оружие'], 'subcategories': ['rifle'], 'category_probabilities': {'Аксессуары и Запчасти': 0.0036617859732359648, 'Снаряжение и защита': 0.0009852994699031115, 'Страйкбольное оружие': 0.9983514547348022}, 'subcategory_probabilities': {'HK': 0.006292761769145727, 'M serias': 0.006835001055151224, 'ak': 0.00799906812608242, 'backpack': 0.011783991940319538, 'helmet': 0.0028981147333979607, 'mashinegun': 0.00692513445392251, 'pistol': 0.0034643891267478466, 'pouch': 0.0049142478965222836, 'rifle': 0.9984733462333679, 'shutgun': 0.030544739216566086, 'vest': 0.0042833611369132996}}


In [7]:
result = predict_with_tta('тест\мка ар.webp', model)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
{'categories': ['Страйкбольное оружие'], 'subcategories': ['M serias'], 'category_probabilities': {'Аксессуары и Запчасти': 0.0018220817437395453, 'Снаряжение и защита': 0.0002366189582971856, 'Страйкбольное оружие': 0.999935507774353}, 'subcategory_probabilities': {'HK': 0.05416689068078995, 'M serias': 0.5270873308181763, 'ak': 0.0765266865491867, 'backpack': 0.011207496747374535, 'helmet': 0.002302464796230197, 'mashinegun': 0.0039348965510725975, 'pistol': 0.0019907939713448286, 'pouch': 0.00211693299934268, 'rifle': 0.023626241832971573, 'shutgun': 0.010622268542647362, 'vest': 0.0013050779234617949}}
